# Automated ML  

> All the dependencies needed to complete the project are listed below

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resourceses

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory

# Dependencies required to create or attach AmlCompute cluster:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.pipeline.steps import AutoMLStep

# needed to display the run details
from azureml.widgets import RunDetails

import joblib
# Needed for the deployment part
from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

> I will bring at this stage the creation of the experiment and the creation / attachment of the AmlCompute to the workspace.  
> Doing so, I will keep the same approach that was suggested during the second Udacity project. 


In [ ]:
experiment_name = 'heart-failure-experiment-automl'
project_folder = './capstone-project'

experiment = Experiment(ws, experiment_name)
experiment

# starting an interactive logging session, as recommended in Azure documentation 'how-to-log-view-metrics'
run=experiment.start_logging()

In [ ]:
# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# I am using here the get_status() for a more detailed view of current AmlCompute status:
print(compute_target.get_status().serialize())

## Dataset

### Overview

I will use [Kaggle](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) "Heart Failure Prediction dataset".  
This dataset is related to a study that  focused on survival analysis of 299 heart failure patients who were admitted to Institute   
of Cardiology and Allied hospital Faisalabad-Pakistan during April-December (2015).   
All the patients were aged 40 years or above, having left ventricular systolic dysfunction.  

The dataset contains the following 12 clinical features, plus one target feature ("death event"):  
A data analysis report is available onmy github repo, [here](https://github.com/JCForszp/nd00333-capstone/blob/master/Datasets/heart%20failure%20report.html)

**Clinical features:**
- age: age of the patient (years)
- anaemia: decrease of red blood cells or hemoglobin (boolean)
- high blood pressure: if the patient has hypertension (boolean)
- creatinine phosphokinase (CPK): level of the CPK enzyme in the blood (mcg/L)
- diabetes: if the patient has diabetes (boolean)
- ejection fraction: percentage of blood leaving the heart at each contraction (percentage)
- platelets: platelets in the blood (kiloplatelets/mL)
- sex: woman or man (binary)
- serum creatinine: level of serum creatinine in the blood (mg/dL)
- serum sodium: level of serum sodium in the blood (mEq/L)
- smoking: if the patient smokes or not (boolean)
- time: follow-up period (days)

**Target feature:**
- [target] death event: if the patient deceased during the follow-up period (boolean)

We are dealing here with a classification task, i.e trying to predict the outcome of the follow-up period based on the given clinical features.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
found = False
key = "JCF-heart-failure-dataset"
description_text = "Kaggle Heart Failure Prediction dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://github.com/JCForszp/nd00333-capstone/blob/master/Datasets/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
print(f"Accuracy obtained by predicting the most frequent value ("null acccuracy", as baseline) : {df['DEATH_EVENT'].value_counts().head(1)/len(df['DEATH_EVENT'])}.")

## AutoML Configuration

### Note on automl settings selection:  

- **n_cross_validations**: 4,
10 is a usual value for cross-validations, but the size of the dataset is relatively small.  
Hence, a 90/10% split seems a bit disproportionate. I prefer to take a 75/25% split,  
which will end of with testing sets of 75 patients, so probably more reasonable and keeping the  
number of runs low. 
- **primary_metric**: 'accuracy',  
'accuracy' is the most frequent and easiest metrics to use for classification tasks  
- **enable_early_stopping**: True,  
According to [Azure documentation](https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py), this  
settings allows automl to terminate a score determination if the score is not improving.  
The default value is 'False' and hence, needs to be set to 'True' at config level.  
Microsoft documentation mentions that *Early stopping window starts on the 21st iteration  
and looks for early_stopping_n_iters iterations (currently set to 10).  
This means that the first iteration where stopping can occur is the 31st.*    
Hence, this setting is a nice to have, but won't be critical for our limited exercice. 
- **max_concurrent_iterations**: 4,  
According to Microsoft documentation *Represents the maximum number of iterations that would be executed in parallel.  
The default value is 1.*  
In our compute_config, we chose a value of 4, and the number of concurrent values needs to be less or equal to that number.  
Hence, the value of this setting. 
- **experiment_timeout_minutes**: 20,    
Defines how long, in minutes, the experiment should continue to run. 
Looking at Azure documentation on [how-to-configure-auto-train](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train),  
20mn seemed to be a reasonable trade-off.  
- **verbosity**: logging.INFO  
The verbosity level for writing to the log file. The default is INFO or 20. So, we could basically have skipped this setting, but it seems  
good practice to specify it every time, to assess if it's really the optimal level of detail.

### Note on automl_config settings:
- **compute_target** = compute_target,  
This is the Azure Machine Learning compute target to run the Automated Machine Learning experiment on.   
It corresponds to the compute_target we defined above in the script, right after the import of the dependencies.  
- **task**='classification',  
Three types of tasks are allowed here:'classification', 'regression', or 'forecasting'.  
As mentioned in the Dataset section, we are clearly here in a classification task. 
- **training_data**=dataset,  
This is the dataset we registered in previous cell. 
- **label_column_name**='DEATH_EVENT',  
This is the name of the target column. The original dataset on Kaggle clearly defines 'DEATH_EVENT' as being the label column.  
- **path** = project_folder,  
We set this project_folder to './capstone-project'  
- **featurization**= 'auto',  
Two values allowed: 'auto' and 'off'. Based on Microsoft doc, setting featurization to off  would mean re-doing manually    
all one-hot encoding, managing missing values,... It meakes total sense to leave automl dealing with that on a pre-cleaned dataset. 
- **debug_log** = "automl_errors.log",  
The log file to write debug information to. If not specified, 'automl.log' is used.  
I just set the name to one I chose.  
- **enable_onnx_compatible_models**=False,  
ONNX is presented as a way to optimize the inference of the ML model.[(doc)](https://docs.microsoft.com/en-us/azure/machine-learning/concept-onnx)   
We are dealing with a small-sized dataset, so I chose to leave this setting of False and I will investigate this feature separately later.   
- **automl_settings**  
Brings the automl_settings dictionary we defined above in the automl_config object. 

In [ ]:
# TODO: Put your automl settings here
automl_settings = {"n_cross_validations": 5,
                    "primary_metric": 'accuracy',
                    "enable_early_stopping": True,
                    "max_concurrent_iterations": 4,
                    "experiment_timeout_minutes": 20,
                    "verbosity": logging.INFO
                    }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                            compute_target = compute_target,
                            task='classification',
                            training_data=dataset,
                            label_column_name='DEATH_EVENT',
                            path = project_folder,
                            featurization= 'auto',
                            debug_log = "automl_errors.log",
                            enable_onnx_compatible_models=False,
                            **automl_settings
                            )

In [ ]:
# Experiment Submission
# Theshow_output parameter switches on the verbose logging
my_run = experiment.submit(automl_config, show_output = True)
# We use the same parameter in the wait_for_completion function on the resulting run.
my_run.wait_for_completion(show_output = True)

In [ ]:
# This command fetches the run status and displays it in this notebook as confirmation. 
print("Run Status: ",my_run.get_status())

## Run Details

>We  use here the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(my_run).show()

In [ ]:
my_run.wait_for_completion()

In [ ]:
print("AutoML Run Summary: ", my_run.summary())

## Best Model

In the cell below, we get the best model from the automl experiments and display all the properties of the model.



### Retrieval of best model from the automl experiments

In [ ]:
best_run, fitted_model = my_run.get_output() # Return the run with the corresponding best pipeline that has already been tested.
                                             # as we do not mention any parameter, get_output returns the best pipeline according to the primary metric ('accuracy'.
best_run

### Display of all the properties of the best model

In [ ]:
# we display, below, the metrics, details and properties (this is the order that makes most sense)

print('*'*50)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name,":" , metric)

print('*'*50)
print("Best run details :",best_run.get_details())

print('*'*50)
print("Best run properties :",best_run.get_properties())
print('*'*50)

In [ ]:
print(fitted_model)

### Saving the best model

In [ ]:
os.makedirs('outputs', exist_ok=True)


In [ ]:
model = best_run.register_model(
                                workspace=ws,
                                model_name = 'best_run', 
                                model_path = './outputs/model.pckl',
                                )

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
best.run.get_file_names()

In [ ]:
best_run.download_file('outputs/scoring_file_v_1_0_0.py','score.py')
# automl_best_run.download_file('outputs/conda_env_v_1_0_0.yml','env.yml')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
# Creating an inference config
inference_config = InferenceConfig(
                                    entry_script="score.py",
                                    environment=best_run.get_environment()
                                  )

# Deploying the model as a web service to an Azure Container Instance (ACI)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service_name = 'heartfprediction'
webservice = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

webservice.wait_for_deployment(show_output=True)

In [ ]:
print(webservice.state)

In [ ]:

primary_key, secundary_key = webservice.get_keys()
print("Service State: ", webservice.state)
print("Scoring URI: ",   webservice.scoring_uri)
print("Swagger URI: ",   webservice.swagger_uri)
print("Primary key: ",primary_key)


In [ ]:
df = df.drop(columns=["DEATH_EVENT"])

input_data = json.dumps({
                        'data': df.sample(10).to_dict(orient='records')
                        })

# I sent a random sample and expect a proportion of negative death event ('0') between 5 and 7 or 8,   
# based on the dataset proportion

In [ ]:
scoring_uri = webservice.scoring_uri
input_data = data_sample

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
response = requests.post(scoring_uri, input_data, headers=headers)
response.json()

In [ ]:
for case in range(0,10):
    print(f"Case details: {data['data'][case]}")
    print(f"Death event: {'yes' if response.json()[case]==1 else 'no'}")
    print('*'*20)
          

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
webservice.get_logs()

In [ ]:
webservice.delete()